<a href="https://colab.research.google.com/github/Sivani-Bammidi/Emotion-detection/blob/main/Copy_of_Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from zipfile import ZipFile
file_name = "archive.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
print("Done")

Done


In [35]:
from tensorflow.python import image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [24]:
train_dir = 'train'
val_dir = 'test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical')

valadition_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [25]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [30]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit(train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=valadition_generator,
        validation_steps=7178 // 64)

Epoch 1/50
448/448 [==============================] - 12s 26ms/step - loss: 1.1841 - accuracy: 0.5537 - val_loss: 1.1890 - val_accuracy: 0.5403
Epoch 2/50
448/448 [==============================] - 11s 24ms/step - loss: 1.1579 - accuracy: 0.5649 - val_loss: 1.1762 - val_accuracy: 0.5516
Epoch 3/50
448/448 [==============================] - 11s 25ms/step - loss: 1.1311 - accuracy: 0.5765 - val_loss: 1.1551 - val_accuracy: 0.5569
Epoch 4/50
448/448 [==============================] - 11s 24ms/step - loss: 1.1039 - accuracy: 0.5842 - val_loss: 1.1407 - val_accuracy: 0.5624
Epoch 5/50
448/448 [==============================] - 11s 24ms/step - loss: 1.0850 - accuracy: 0.5943 - val_loss: 1.1358 - val_accuracy: 0.5667
Epoch 6/50
448/448 [==============================] - 11s 25ms/step - loss: 1.0581 - accuracy: 0.6043 - val_loss: 1.1290 - val_accuracy: 0.5693
Epoch 7/50
448/448 [==============================] - 12s 26ms/step - loss: 1.0423 - accuracy: 0.6110 - val_loss: 1.1184 - val_accuracy:

In [31]:
emotion_model.save('model.h5')

In [32]:
from keras.models import load_model
emotion_model = load_model('model.h5')

In [36]:
def emotion_analysis(emotions):
  objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
  y_pos = np.arange(len(objects))

  plt.bar(y_pos, emotions, align='center', alpha=0.5)
  plt.xticks(y_pos, objects)
  plt.ylabel('percentage')
  plt.title('emotion')
  plt.show()

In [38]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript(''' async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename